<a href="https://colab.research.google.com/github/sladem-tox/Resbaz/blob/main/Participant_ChemInf_QSAR_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 0: Demonstration of Cheminformatic and QSAR workflow for ResBaz
`Dr. Slade Matthews, The University of Sydney, 2023`

Computational methods in pharmacology and toxicology have matured!

<img src="https://github.com/sladem-tox/Resbaz-slade/blob/main/ChemResTox_AI_Cover2023.jpg?raw=true" alt="ChemResTox Cover" width="600" height="700"/>

In today's workshop demonstration we will take a group of molecules from a publihsed paper and try to model the activity relationship contained in the experimental data.

<b>Today we will perform the following steps:</b>

1.   Get chemical data from Github
2.   Calculate chemical fingerprints from SMILES representations
3.   Generate model to predict psychedelic activity

The paper we are looking at is
<b>Keshavarz, M.H., Z. Shirazi, and M.A. Rezayat, A simple method for assessing the psychotomimetic activity of the substituted phenethylamines. Zeitschrift für anorganische und allgemeine Chemie, 2021. 647(6): p. 651-662.</b>








<img src="https://github.com/sladem-tox/Resbaz-slade/blob/main/Keshharvarz_masthead.jpg?raw=true" alt="Keshharvarz Masthead" width="800"/>

From this paper we get a list of phenethylamines and their associated activity values expressed as LogA (LogMU) which is the log of the activity in mescaline units. This is defined as the ratio of the effective dose of mescaline to the effective dose of the tested compound (See Thakur et al., 2003*). Meaning that a large value indicates more potency (because the effective mescaline dose is divided by a smaller value). Of course some studies use binding values for 5HT-2A receptors rather than effective hallucinogenic doses but this seems like more fun.

<img src="https://github.com/sladem-tox/Resbaz-slade/blob/main/Psychedelic_Drugs.jpg?raw=true" alt="Psychedelic Drugs" width="600"/>

Image Source: https://www.axios.com/2023/06/26/fda-guidance-psychedelic-drugs-lsd-mushrooms

# Chapter 1: Get the Data

In [ ]:
#First we download our data from github
#Although Colab can mount your GoogleDrive it is a pain and actually quicker to access files from your Github account.
import pandas as pd
df =pd.read_csv("https://github.com/sladem-tox/Resbaz-slade/raw/main/molecule_Phenethylamines_valid.csv")

In [ ]:
######
######

In [ ]:
!pip install rdkit

In [ ]:
#Add a molecule column and make sure RDkt can convert all SMILES
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
######
######

In [ ]:
#Check for smiles that rdkit can't convert to molecule. If sum = 0 then they are all OK
######

In [ ]:
#Define function to generate fp's from SMILES
#Here we are producing a Morgan FP with radius 2 and calling for 1024 bits.
import numpy as np
def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 2, nBits=1024)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

In [ ]:
# Demonstrate that the function "mol2fp" is working with a single SMILES.
######
######

In [ ]:
# Now use the mol2fp function to genereate fingerprints for all rdkit molecule objects.
# Here we are creating a new column called FPs in the dataframe df and applying the mol2fp function to it.
######
######

# Chapter 2: About fingerprints

<img src="https://github.com/sladem-tox/Resbaz-slade/blob/main/FingerPrint.jpg?raw=true" alt="Fingerprint" width="550"/>

A chemical fingerprint is an identifying description of a molecule created by interogating the molecule and listing its features. The list of features is encoded as a series of "bits" which take the value one if the feature is present and zero if the molecule doesn't have the feature corresponding to that bit in the list.

HERE is a fantastic video by Greg Landrum the author of rdkit explaining how it works and what fingerprints are.
https://www.youtube.com/watch?v=ERvUf_lNopo&t=13s

Also note that there are lots of ways to generate fingerprints. Here is a link to a tool that can generate some other types beyond those available in rdkit.
https://github.com/hcji/PyFingerprint

HERE are a couple of the code examples Greg used in his video:


In [ ]:
# Take a few molecule objects from our dataframe
mol_list = df['Molecule'].head(3).tolist()

In [ ]:
# Now we can draw these and inspect them.
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
######
######

In [ ]:
# Now take the second molecule, 2,5-dimethoxy-4-chloroamphetamine, which we will call Dimethoxy4C for short.
######

In [ ]:
# Here we generate a fingerprint for Dimethoxy4C and display each "bit" that is "on"
bi = {}

fp = AllChem.GetMorganFingerprintAsBitVect(Dimethoxy4C, 2, nBits=1024, bitInfo=bi)
fp_arr = np.zeros((1,))
DataStructs.ConvertToNumpyArray(fp, fp_arr)
np.nonzero(fp_arr)
list(fp.GetOnBits())

In [ ]:
# Here we draw each part of Dimethoxy4C corresponding to the "on" bits.
######
######

In [ ]:
# Now we want to extract the fingerprints into seperate columns for modelling.
######
######

In [ ]:
#Bring back the outcome column
fp_df.insert(1024, "LogA", df["LogA"])
fp_df.head(2)

In [ ]:
# Bring back the name column for later use
fp_df.insert(0, "Name", df["Name"])
fp_df.head(2)

# Chapter 3: Now we want some training sets and test sets and do some modelling

In [ ]:
#Load data from pandas dataframe
######
######

In [ ]:
# Show first two lines of x
######

In [ ]:
#What is the shape of y?
######

In [ ]:
# sklearn has a nice tool for setting up training and testing sets in data modelling projects.
# Here we are splitting the data into 85% training data and 15% test data where x indicates the set of descriptor variables
# and y indicates the set of target values i.e. logA in this case.

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import set_config

######

## About Models

There are an infinite amount of ways to find the mathematical relationship between chemical descriptors and biological activity.
Here we will consider some regression models available in the Scikitlearn Python library. You might want to develop a more sophisticated model such as a neural network in which case I would suggest checking out Pytorch:
https://pytorch.org/


In [ ]:
# Since the logA is a scalar variable (non-binary) we want to do some kind of regression to model it.
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

######
######

In [ ]:
print("Training Performance Statistics")
print("-------------------------------")
score = rfr.score(xtrain, ytrain)
print("R-squared:", score)

######

mse = mean_squared_error(ytest, ypred)
print("MSE: ", mse)
print("RMSE: ", mse**(1/2.0))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

######
######
######

#Find axis dimensions for scatter plot
import math
max_dim = math.ceil(max(x.max(),y.max())) +1
min_dim = math.floor(min(x.min(), y.min())) -1

plt.scatter(x, y)
plt.ylim([min_dim, max_dim])
plt.xlim([min_dim, max_dim])
plt.plot(x, a*x+b)
plt.title("Phenethylamine Psychedelic Activity Prediction")
plt.xlabel('Test Set Log A Values')
plt.ylabel('Predicted LogA Values')
# The optional line below saves the figure to file as an eps vector graphic as required in many journals.
# If you can't open in try using Acrobat, it will be able to read the postcript.
plt.savefig('phenethylamine_predictions.eps', format='eps')


In [ ]:
print("Testset Prediction Performance")
print("------------------------------")
correlation = np.corrcoef(x, y)[0,1]
print("Correlation:", correlation)
#R-square for Test Set results (above is R-square for training results)
rsquare = correlation**2
print("Testset Rsquare:", rsquare)

# Chapter 4: What if I want to use the model later to predict how trippy our new molecule is?

In [ ]:
# joblib can be used to save and load trained models.
# This is called pickelling the model so they are called dot pkl files.
import joblib

######

In [ ]:
# load the model
model = joblib.load('/content/psychedelics.pkl')

In [ ]:
# Here we use the mol2fp function from above to convert a smiles string into a morgan fingerprint with radius 2.
test_smi = ("CCCNCCCCC")

######
######


In [ ]:
# Finally, we use model.predict to output a predicted value for logA.

######

# Chapter 5: Afterward

Of course all the code above is only a demonstration and you would need to do a whole lot more development and testing before you would finish up with a reliable predictive model. The reality is that data curation and question formulation are 90% of the work and the modelling is the fun part! There are many ways to represent molecules (different fingerprints, chemcial descriptors, DFT parameters) and many ways to represent the mathematical relationships between structure and activity, and finally, many ways to test the predictive capability of a new model. What we have learned, however, is how powerful cloud-based resources can be used for demonstrating and sharing computing techniques and we have learned a few nice Python tricks for cheminformatic analysis.

## Bonus material!
Another cool thing that you can do in Google colab with rdkit and Python is use it to draw (as seen above) and identify molecules. Here is some code to interrogate a smiles string and find out the name of the molecule by accessing PubChem online. Things like this and the molecule drawing code are handy to keep in a google colab sheet in your Google drive for molecule rangling!

In [ ]:
# First we install pubchempy
!pip install pubchempy

In [ ]:
# Here is an example smiles structure we might be given to investigate
SMILES = "CN1[C@H]2CC[C@@H]1[C@H]([C@H](C2)OC(=O)C3=CC=CC=C3)C(=O)OC"

What have they given us? From the code above we learned how to draw a molecule from smiles using rdkit. Let's draw our new molecule here. Can you tell what we have been given from the molecular structure?

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

######
######

Maybe you can... but many would not be able to identify this structure. So let's see what we have here by asking PubChem.

In [ ]:
import pubchempy as pcp

# Get the compound information by SMILES

######

# Check if any compounds were found
if drug:
    # Extract the compound ID from the format "[compound(3715)]"
    compound_id = drug[0].to_dict()['cid']

    # Use the compound ID to fetch compound information
    c = pcp.Compound.from_cid(compound_id)

    # Check if the compound information was retrieved successfully
    if c:
        # Print synonyms for the compound
        print(c.synonyms)
    else:
        print(f"Compound with CID {compound_id} not found.")
else:
    print("No compounds found for the given SMILES string.")


# References

`Keshavarz, M.H., Z. Shirazi, and M.A. Rezayat, A simple method for assessing the psychotomimetic activity of the substituted phenethylamines. Zeitschrift für anorganische und allgemeine Chemie, 2021. 647(6): p. 651-662.`

`Thakur, M., A. Thakur, and P.V. Khadikar, QSAR studies on psychotomimetic phenylalkylamines. Bioorg Med Chem, 2004. 12(4): p. 825-31.`

`Greg Landrum's video: https://www.youtube.com/watch?v=ERvUf_lNopo&t=13s`
